<a href="https://colab.research.google.com/github/sanj1210/PALM/blob/main/PALM_2_COT_text_bison_decimal_numbers_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install google-cloud-aiplatform==1.36.2 --upgrade --user
!pip uninstall bigframes
!pip install bigframes==0.26.0

import bigframes.dataframe
import IPython
import time
from datasets import load_dataset, concatenate_datasets
import re
import pandas as pd
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.language_models import (
    TextGenerationModel,
    TextEmbeddingModel,
    ChatModel,
    InputOutputTextPair,
    CodeGenerationModel,
    CodeChatModel,
)

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

import vertexai

PROJECT_ID = "primeval-rain-437401-c8"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


dataset_name = "asdiv"  # Change this to "addsub" or "mawps" or "asdiv" as needed

# Load dataset based on the chosen name
def load_dataset_by_name(name):
    if name == "asdiv":
        dataset = load_dataset("EleutherAI/asdiv")
        eval_data = pd.DataFrame(data=dataset["validation"])
        eval_data["Full_Question"] = eval_data["body"] + ". " + eval_data["question"]
        eval_data["Int_Answer"] = eval_data["answer"].apply(lambda x: x.strip().split(" ")[0])
    elif name == "addsub":
        dataset = load_dataset("allenai/lila", data_dir="addsub")
        lila_combined = concatenate_datasets([dataset['train'], dataset["validation"], dataset['test']])
        eval_data = pd.DataFrame(data=lila_combined)
        eval_data = eval_data[eval_data["dataset"].str.contains("addsub")]
        eval_data["Full_Question"] = eval_data["output_program"]
        eval_data["Int_Answer"] = eval_data["result_float"]
    elif name == "mawps":
        dataset = load_dataset("MU-NLPC/Calc-mawps")
        eval_data = pd.DataFrame(data=dataset["validation"])
        eval_data["Full_Question"] = eval_data["question"]  # Adjust based on actual column names
    else:
        raise ValueError("Unknown dataset name!")

    return eval_data

# Load the selected dataset
eval_data = load_dataset_by_name(dataset_name)

# Define functions for answer extraction
def extract_answer(output):
    try:
        split_output = output.split("The answer is")
        if len(split_output) > 1:
            answer_part = split_output[-1]
        else:
            answer_part = output
        match_num = re.search(r'((?:\d{1,3},(?:\d{3},)*\d{3}|\d+)(?:\.\d+)?)(\.)?', answer_part)
        if match_num:
            answer = match_num.group(1).replace(",", "")
            return float(answer)
        return "No answer"
    except Exception as e:
        print("Error extracting answer:", e)
        return "No answer"

def extract_ground_truth_value(ground_truth_ans):
    try:
        if isinstance(ground_truth_ans, str):
            match_num = re.search(r'\d+(\.\d+)?', ground_truth_ans)
            if match_num:
                return float(match_num.group())
        return "No answer"
    except Exception as e:
        print("Error extracting ground truth value:", e)
        return "No answer"

# Initialize the model
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

# Define exemplar prompts
exemplars = '''
Think step by step to answer the given question. The last sentence of the answer must be of the format `The answer is (x)`.
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8.

'''

# Function to process a batch of questions
def process_batch(batch):
    responses = []
    for question in batch:
        input_prompt = exemplars + "\n" + "Q: " + question + "\n" + "A: "
        try:
            response = generation_model.predict(prompt=input_prompt, max_output_tokens=1024, temperature=0.2, top_k=0.8, top_p=0.4)
            responses.append(response.text)
        except Exception as e:
            print("Exception caught:", e)
            responses.append("Error: Quota Exceeded")
        time.sleep(2)  # Adding a delay between requests
    return responses

# Define the range of questions to process
start_index = 0
end_index = len(eval_data) - 1
#subset_eval = eval_data.iloc[start_index:end_index + 1]
subset_eval = eval_data.iloc[start_index:10]

# Process the subset of the dataset in batches (each batch processes 5 questions)
batch_size = 5
results = []
questions_to_process = subset_eval['Full_Question']

for i in range(0, len(questions_to_process), batch_size):
    batch_questions = questions_to_process[i:i + batch_size]
    batch_responses = process_batch(batch_questions)
    results.extend(batch_responses)
    print(f"Processed batch {i // batch_size + 1}")
    time.sleep(47)

# Create a dataframe for results
ids = list(range(start_index, start_index + len(subset_eval)))
if dataset_name == "asdiv":
    ground_truth_answers = subset_eval['Int_Answer']
elif dataset_name == "mawps":
    ground_truth_answers = subset_eval['Int_Answer']
else:
    ground_truth_answers = subset_eval['output_answer']
extracted_values = [extract_answer(response) for response in results]
ground_truth_values = [extract_ground_truth_value(answer) for answer in ground_truth_answers]

# Calculate correctness
correctness = []
for extracted, ground_truth in zip(extracted_values, ground_truth_values):
    correctness.append(1 if extracted == ground_truth else 0)

# Creating a results dataframe
results_df = pd.DataFrame({
    'id': ids,
    'question': subset_eval['Full_Question'],
    'groundTruthAns': ground_truth_answers,
    'model output': results,
    'extracted value': extracted_values,
    'ground truth value': ground_truth_values,
    'correctness': correctness
})

# Save to a CSV file
results_df.to_csv(f"responses_{dataset_name}.csv", index=False)

# Function to compute accuracy and save results
def compute_accuracy_and_save(results_df, dataset_name, model_name, prompt_type):
    accuracy = results_df['correctness'].mean()
    output_content = f"{{Dataset: {dataset_name}, \nmodel_name: {model_name}, \naccuracy: {accuracy}, \nprompt: {prompt_type}}}"

    with open(f"accuracy_{dataset_name}.txt", "w") as f:
        f.write(output_content)

    if "google.colab" in sys.modules:
        from google.colab import files
        files.download(f"accuracy_{dataset_name}.txt")
        files.download(f"responses_{dataset_name}.csv")

# Compute accuracy and save results
compute_accuracy_and_save(results_df, dataset_name, "text-bison@001", "COT")


ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.9 MB/s eta 0:00:00
